최종 테이블 생성 알고리즘 추가
- 주문ID 수정
- Volume 수정
- final_dispatch 수정
- sequence 수정
- 최종 테이블 전역화


In [1]:
from pathlib import Path
import pandas as pd

# import pandas_profiling as pp

In [2]:
def load_data():
    data_dir = Path("data")
    df_veh_table = pd.read_csv(data_dir / "veh_table.csv")
    df_terminals = pd.read_csv(data_dir / "Terminals.csv", encoding='euc-kr')
    df_orders_table = pd.read_csv(data_dir / "orders_table.csv", encoding='euc-kr')
    df_od_matrix = pd.read_csv(data_dir / "od_matrix.csv")

    return df_veh_table, df_terminals, df_orders_table, df_od_matrix

df_veh_table, df_terminals, df_orders_table, df_od_matrix = load_data()

In [18]:
df_od_matrix

,origin,Destination,Distance_km,Time_minute
0,D_69,D_69,0.000000,0.000000
1,D_69,D_1916,6.912456,11.015591
2,D_69,D_562,9.105672,14.232512
3,D_69,D_12,14.798563,21.089274
4,D_69,D_1587,20.435462,32.083726
...,...,...,...,...
2214139,D_68,D_1605,380.623149,279.612434
2214140,D_68,D_224,402.319443,281.005369
2214141,D_68,D_528,387.712806,281.732369
2214142,D_68,D_1610,403.059343,281.752449


In [13]:
def load_data():
    data_dir = Path("data")
    df_final = pd.read_csv(data_dir / "final.csv", index_col=0)

    return df_final

df_final = load_data()
df_final

,주문ID,하차지_위도,하차지_경도,착지ID,CBM,하차가능시간_시작,하차가능시간_종료,하차작업시간(분),터미널ID,date,Group,length(meter)_x,time(min)_x,가장 가까운 터미널,클러스터링 터미널 (kmeans),부군집
0,20230501B464859_001,36.448483,127.409240,D_1,27.893621,18:00,2:00,60,O_179,2023-05-01,0,45.598193,59.162692,O_144,O_144,NaN
1,20230501B464914_001,36.448483,127.409240,D_1,42.669815,18:00,2:00,60,O_673,2023-05-01,1,168.798711,183.776486,O_144,O_144,NaN
2,20230501B464917_001,36.448483,127.409240,D_1,27.343155,18:00,2:00,60,O_673,2023-05-01,2,168.798711,183.776486,O_144,O_144,NaN
3,20230501B464927_001,36.448483,127.409240,D_1,19.722170,18:00,2:00,60,O_179,2023-05-01,3,45.598193,59.162692,O_144,O_144,NaN
4,20230501B464944_001,36.448483,127.409240,D_1,28.031678,18:00,2:00,60,O_673,2023-05-01,0,168.798711,183.776486,O_144,O_144,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15787,PND000304,36.639999,127.463585,D_2902,19.244336,10:00,17:00,60,O_580,2023-05-01,1,103.840751,136.427834,O_179,O_629,NaN
15788,PND000380,37.342094,127.859909,D_2917,23.038265,18:00,2:00,60,O_349,2023-05-05,1,134.242123,191.359092,O_580,O_580,NaN
15789,PND000469,37.332897,129.070282,D_2965,18.741028,00:00,6:00,60,O_553,2023-05-03,0,64.614928,86.870512,O_553,O_553,NaN
15790,PND000498,37.654900,127.236572,D_1567,28.702799,00:00,6:00,60,O_349,2023-05-04,2,139.831222,190.208865,O_174,O_62,NaN


In [4]:
from datetime import datetime, timedelta
import math

def str_to_datetime(str_date, batch):
    time_components = [0, 6, 12, 18]
    time = timedelta(hours=time_components[batch])
    temp = datetime.strptime(str_date, '%Y-%m-%d') + time
    return temp

def string_to_timedelta(time_str):

    time_format = '%H:%M'
    time_obj = datetime.strptime(time_str, time_format)
    time_delta = timedelta(hours=time_obj.hour, minutes=time_obj.minute)
    return time_delta

def timewindow_to_datetime(date, timewindow):
    # timewindow 자료형 변경
    timewindow = (string_to_timedelta(timewindow[0]), string_to_timedelta(timewindow[1]))
    date_timedelta = timedelta(hours=date.hour, minutes=date.minute)

    timewindow_list = [0, 0]
    # timewindow_start < batch < timewindow_end 경우, 마지막날의 timewindow_start~batch 반영되지 않음
    if date_timedelta < timewindow[1]:  # 배치 시간보다 타임 윈도우 끝나는 시간이 더 크면 (batch < timewindow_end)
        if timewindow[1] > timewindow[0]:   # 타임 윈도우 끝나는 시간이 시작하는 시작보다 크면 (timewindow_end > timewindow_start)
            if timewindow[0] < date_timedelta < timewindow[1]:  # timewindow_start < batch < timewindow_end
                timewindow_list[1] = date + timedelta(hours=72)
                timewindow_list[0] = timewindow_list[1] - (date_timedelta - timewindow[0])

                timewindow_tuple = (timewindow_list[0], timewindow_list[1])
                return timewindow_tuple

            else:
                timedelta_temp_1 = timedelta(hours=48) + timewindow[1] - date_timedelta
                timedelta_temp_0 = timedelta_temp_1 - timewindow[1] + timewindow[0]
        else:   # 타임 윈도우 끝나는 시간이 시작하는 시작보다 작으면 (timewindow_end <= timewindow_start)
            timedelta_temp_1 = timedelta(hours=72)
            timedelta_temp_0 = timedelta(hours=48) + timewindow[0] - date_timedelta

    else:   # 배치 시간보다 타임 윈도우 끝나는 시간이 더 작으면 (batch >= timewindow_end)
        if timewindow[1] > timewindow[0]:   # 타임 윈도우 끝나는 시간이 시작하는 시작보다 크면 (timewindow_end > timewindow_start)
            timedelta_temp_1 = timedelta(hours=72) - date_timedelta + timewindow[1]
            timedelta_temp_0 = timedelta_temp_1 - timewindow[1] + timewindow[0]
        else:   # 타임 윈도우 끝나는 시간이 시작하는 시작보다 작으면 (timewindow_end <= timewindow_start)
            timedelta_temp_1 = timedelta(hours=72) - date_timedelta + timewindow[1]
            timedelta_temp_0 = timedelta(hours=48) + timewindow[0] - date_timedelta

    timewindow_list[1] = date + timedelta_temp_1
    timewindow_list[0] = date + timedelta_temp_0

    timewindow_tuple = (timewindow_list[0], timewindow_list[1])
    return timewindow_tuple

def calc_timewindow_duration(timewindow):
    timewindow = (string_to_timedelta(timewindow[0]), string_to_timedelta(timewindow[1]))
    if timewindow[0] <= timewindow[1]:
        duration = timewindow[1] - timewindow[0]
    else:
        duration = timedelta(hours=24) - timewindow[0] + timewindow[1]

    return duration

class Vehicle:  # 차량 번호, 최대적재량, 현위치, 고정비용, 운행비용, 배차여부, (총 운영비)
    def __init__(self, veh_num, max_load, cur_loc, fixed_cost, variable_cost, dispatched_time=0, dispatched=0):
        self.veh_num = veh_num
        self.max_load = max_load
        self.cur_loc = cur_loc
        self.fixed_cost = fixed_cost
        self.variable_cost = variable_cost
        self.dispatched_time = dispatched_time
        self.dispatched = dispatched

    @classmethod
    def get_unique_max_load(cls):
        unique_max_load = set(instance.max_load for instance in cls.instances)
        return unique_max_load

# 주문ID, 착지ID, CBM(적재량), time window(하차가능시간_시작&종료), 물품 위치(터미널ID), 주문 발생 날짜, 배치 번호, length(meter)_x, time(min)_x, 소속 군집
class Order:

    def __init__(self, order_id, arrival_id, cbm, timewindow, item_loc, date, batch, length, time, cluster, delivered=0):
        self.order_id = order_id
        self.arrival_id = arrival_id
        self.cbm_former = cbm
        self.cbm = math.ceil(cbm)
        self.item_loc = item_loc
        self.date = str_to_datetime(date, batch)
        self.batch = batch
        self.length = length
        self.time = time
        self.cluster = cluster
        self.delivered = delivered
        self.timewindow_former = timewindow
        self.timewindow = timewindow_to_datetime(self.date, timewindow)
        self.timewindow_duration = calc_timewindow_duration(self.timewindow_former)

    @classmethod
    def get_unique_dates(cls):
        unique_dates = set(instance.date for instance in cls.instances)
        return unique_dates

    @classmethod
    def get_unique_timewindows(cls):
        unique_timewindows = set(instance.timewindow for instance in cls.instances)
        return unique_timewindows

In [5]:
# 차량 인스턴스 생성
# veh_num, max_load, cur_loc, fixed_cost, variable_cost
vehicles = []
for row in df_veh_table[['VehNum', 'MaxCapaCBM', 'StartCenter', 'FixedCost', 'VariableCost']].itertuples(index=False):
    vehicle = Vehicle(*row)
    vehicles.append(vehicle)

In [6]:
# 주문 인스턴스 생성
# 타임 윈도우 튜플로 묶어주기
df_final['time window'] = list(zip(df_final['하차가능시간_시작'], df_final['하차가능시간_종료']))

# 주문ID, (하차지 위도, 하차지 경도), 착지ID, CBM(적재량), time window(하차가능시간_시작&종료), 물품 위치(터미널ID), 소속 군집
orders = []
for row in df_final[['주문ID', '착지ID', 'CBM', 'time window', '터미널ID', 'date', 'Group', 'length(meter)_x', 'time(min)_x', '클러스터링 터미널 (kmeans)']].itertuples(index=False):
    order = Order(*row)
    orders.append(order)

In [7]:
# 결과 테이블 생성
final_order_table = pd.DataFrame(columns=['주문ID', 'VehicleID', 'Sequence', '착지ID',
                                 'ArrivalTime', 'WaitingTime', 'ServiceTime', 'DepartureTime', 'Delivered'])
final_vehicle_dispatch = pd.DataFrame(columns=['VehNum', 'Count', 'Volume', 'TravelDistance',
                                      'WorkTime', 'TravelTime', 'ServiceTime', 'WaitingTime', 'TotalCost', 'FixedCost', 'VariableCost'])

In [8]:
# 초기 주문 테이블 설정
row_to_add_order = df_final[['주문ID']]
columns_to_insert = ['주문ID']  # 삽입할 column 이름들
for column in columns_to_insert:
    final_order_table[column] = row_to_add_order[column]
final_order_table['Delivered'] = final_order_table['Delivered'].fillna('NO')
final_order_table['주문ID_copy'] = final_order_table['주문ID']
    
row_to_add_order = df_veh_table[['VehNum']]
columns_to_insert = ['VehNum']  # 삽입할 column 이름들
for column in columns_to_insert:
    final_vehicle_dispatch[column] = row_to_add_order[column]
final_vehicle_dispatch.fillna(0, inplace=True)
final_order_table


,주문ID,VehicleID,Sequence,착지ID,ArrivalTime,WaitingTime,ServiceTime,DepartureTime,Delivered,주문ID_copy
0,20230501B464859_001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,20230501B464859_001
1,20230501B464914_001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,20230501B464914_001
2,20230501B464917_001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,20230501B464917_001
3,20230501B464927_001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,20230501B464927_001
4,20230501B464944_001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,20230501B464944_001
...,...,...,...,...,...,...,...,...,...,...
15787,PND000304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,PND000304
15788,PND000380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,PND000380
15789,PND000469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,PND000469
15790,PND000498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,PND000498


In [9]:
# 각 터미널마다 다른 터미널과의 거리 계산하여 정렬 (terminal_distance_dict_total)
terminal_distance_dict_total = {}
for terminal_start in list(df_terminals['ID'].unique()):
    terminal_distance_dict = {}
    for terminal_end in list(df_terminals['ID'].unique()):
        if terminal_end == terminal_start:
            continue
        terminal_distance = df_od_matrix.loc[(df_od_matrix['origin'] == terminal_end) & (df_od_matrix['Destination'] == terminal_start), 'Distance_km'].iat[0]
        terminal_distance_dict[terminal_end] = terminal_distance

    terminal_distance_dict = sorted(terminal_distance_dict.items(), key=lambda item: item[1])
    terminal_distance_dict_total[terminal_start] = terminal_distance_dict

In [10]:
# 군집 내의 주문들의 cbm 합과 배정된 차량의 최대적재량 비교
def comp_orderload_vehmaxload(orders):
    return (sum(order.cbm for order in orders) > 60)    # 해당 타임 윈도우의 모든 주문들의 cbm 합이 60을 넘길 때 배차 알고리즘 실행

# 배송 시 걸리는 시간 계산
# item_loc와 arrival_id 활용하여 od_matrix에서 걸리는 시간 찾아 합하기
def delivery_time(orders, od_matrix, timewindow, batch_date, final_order_table):
    
    # 차량 테이블 최신화를 위한 변수
    total_distance = 0
    orders_time = 0
    wait_time = 0
    travel_time = 0
    service_time = 0

    # 주문 테이블 최신화를 위한 변수
    arrival_time = batch_date
    departure_time = batch_date

    timewindow_met_or_not = True
    prev_arrival_id = orders[0].item_loc

    time_to_terminal = od_matrix.loc[(od_matrix['origin'] == orders[0].item_loc) & (od_matrix['Destination'] == orders[0].arrival_id),'Time_minute'].iat[0]
    if batch_date + timedelta(minutes=time_to_terminal) < timewindow[0][0]:     # 첫 착지 도착했을 때 time window 시작 시간을 지나지 않았으면
        wait_time = timewindow[0][0] - (batch_date + timedelta(minutes=time_to_terminal))
        orders_time += wait_time.total_seconds() / 60
    
    # 대기 시간 최신화
    for order in orders:
        final_order_table.loc[final_order_table['주문ID']
                              == order.order_id, 'WaitingTime'] = 0

    if wait_time != 0:
        final_order_table.loc[final_order_table['주문ID'] ==
                              orders[0].order_id, 'WaitingTime'] = wait_time

    # 최종 주문 테이블 최신화
    # 주문마다 반복
    for order in orders:
        condition_order = final_order_table['주문ID'] == order.order_id

        # 이동 시간 계산
        drive_time = od_matrix.loc[(od_matrix['origin'] == prev_arrival_id) & (
            od_matrix['Destination'] == order.arrival_id), 'Time_minute'].iat[0]

        # 출발시각, 도착시각 최신화
        arrival_time += timedelta(minutes=drive_time)
        departure_time += timedelta(minutes=drive_time)
        if (wait_time != 0) and (orders.index(order) != 0):  # wait_time이 있고 첫 주문이 아닌 경우
            final_order_table.loc[condition_order,
                                  'ArrivalTime'] = arrival_time + wait_time
            departure_time += timedelta(minutes=drive_time) + wait_time
        else:
            final_order_table.loc[condition_order,
                                  'ArrivalTime'] = arrival_time
            departure_time += timedelta(minutes=drive_time)

        orders_time += drive_time
        travel_time += drive_time

        # 도착지가 다른 경우만
        if prev_arrival_id != order.arrival_id:
            # 타임 윈도우 준수 여부
            if batch_date + timedelta(minutes=orders_time) > timewindow[0][1]:
                print("time window를 준수하지 못했습니다.")
                timewindow_met_or_not = False
                break
            final_order_table.loc[condition_order,
                                  'ServiceTime'] = 60  # 상하차시간
            arrival_time += timedelta(minutes=60)   # 도착시각 + 상하차시간
            departure_time += timedelta(minutes=60)   # 출발시각 + 상하차시간
            orders_time += 60
            service_time += 60
        else:
            final_order_table.loc[condition_order, 'ServiceTime'] = 0   # 상하차시간
            # 이동이 없었으니 같은 착지 주문 처리한 것이기 때문에 출발 시각 그대로

        final_order_table.loc[condition_order,
                              'DepartureTime'] = departure_time    # 출발시각 최신화

        prev_arrival_id = order.arrival_id  # 현재 위치 최신화

    last_time = od_matrix.loc[(od_matrix['origin'] == orders[-1].arrival_id) & (
        od_matrix['Destination'] == orders[-1].item_loc), 'Time_minute'].iat[0]
    orders_time += last_time  # 마지막 이동 시간 계산 (착지 -> 터미널)
    travel_time += last_time

    # 이동 거리 측정
    prev_arrival_id = orders[0].item_loc
    for order in orders:
        total_distance += od_matrix.loc[(od_matrix['origin'] == prev_arrival_id) & (
            od_matrix['Destination'] == order.arrival_id), 'Distance_km'].iat[0]
        prev_arrival_id = order.arrival_id

    total_distance += od_matrix.loc[
        (od_matrix['origin'] == orders[-1].arrival_id) & (
            od_matrix['Destination'] == orders[-1].item_loc),
        'Distance_km'
    ].iat[0]

    if not isinstance(wait_time, int):
        wait_time = wait_time.total_seconds() / 60

    return orders_time, timewindow_met_or_not, wait_time, total_distance, travel_time, service_time, final_order_table

# 시간 순으로 정렬 (quick sort)
def order_sort(orders):
    if len(orders) <= 0:
        return orders

    pivot = orders[len(orders) // 2]
    left = [x for x in orders if x.date < pivot.date]
    middle = [x for x in orders if x.date == pivot.date]
    right = [x for x in orders if x.date > pivot.date]

    return order_sort(left) + middle + order_sort(right)

# 차량의 최대적재량 넘지 않게끔 배차
# 추후 가장 많이 담을 수 있게끔 하는 알고리즘 고민해야 함! - '배낭 문제'로 해결
global cbm_over
cbm_over = []

def knapsack_max_cbm(orders, vehicle):

    global cbm_over

    n = len(orders)
    dp = [[0.0 for _ in range(int(vehicle.max_load) + 1)] for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, int(vehicle.max_load) + 1):
            if orders[i - 1].cbm <= j:
                dp[i][j] = max(orders[i - 1].cbm + dp[i - 1][j - int(orders[i - 1].cbm)], dp[i - 1][j])
            else:
                dp[i][j] = dp[i - 1][j]

    # Backtrack to find the orders picked
    picked_orders = []
    i, j = n, int(vehicle.max_load)
    tolerance = 0.001

    while i > 0 and j > 0:
        if abs(dp[i][j] - dp[i - 1][j]) > tolerance:
            picked_orders.append(orders[i - 1])
            j -= int(orders[i - 1].cbm)
        i -= 1

    return picked_orders

# 물품 개수 제한까지 추가한 배낭 문제 알고리즘
def knapsack_max_items_cbm(orders, vehicle, max_items):

    global cbm_over

    n = len(orders)
    dp = [[[0.0 for _ in range(int(vehicle.max_load) + 1)] for _ in range(max_items + 1)] for _ in range(n + 1)]

    for i in range(1, n + 1):
        for k in range(1, max_items + 1):
            for j in range(1, int(vehicle.max_load) + 1):
                if orders[i - 1].cbm <= j and k >= 1:
                    dp[i][k][j] = max(
                        orders[i - 1].cbm + dp[i - 1][k - 1][j - int(orders[i - 1].cbm)],
                        dp[i - 1][k][j]
                    )
                else:
                    dp[i][k][j] = dp[i - 1][k][j]

    # Backtrack to find the orders picked
    picked_orders = []
    i, k, j = n, max_items, int(vehicle.max_load)
    tolerance = 0.001

    while i > 0 and k > 0 and j > 0:
        if abs(dp[i][k][j] - dp[i - 1][k][j]) > tolerance:
            picked_orders.append(orders[i - 1])
            j -= int(orders[i - 1].cbm)
            k -= 1
        i -= 1

    return picked_orders

# 타임 윈도우와 지속시간 합쳐서 유일값 추출
def get_unique_timewindows_combine(instances):
    unique_combinations = set()

    for instance in instances:
        # Combine element1 and element2 into a tuple
        combination = (instance.timewindow, instance.timewindow_duration)

        # Add the combination to the set of unique combinations
        unique_combinations.add(combination)

    return unique_combinations

# 데드라인 얼마 남지 않은 주문이 있는 경우
def orders_shipped_quickly(orders_timewindow, batch_date):
    condition = lambda x: (batch_date - x.date) > timedelta(days=2)

    exists = any(condition(order) for order in orders_timewindow)

    return exists

# 현재 터미널에서 다른 터미널까지의 가까운 거리순으로 정렬
def find_nearest_terminal(cur_terminal, od_matrix, df_terminals):
    terminal_distance_dict = {}
    for terminal in list(df_terminals['ID'].unique()):
        if terminal == cur_terminal:
            continue
        terminal_distance = od_matrix.loc[(od_matrix['origin'] == terminal) & (od_matrix['Destination'] == cur_terminal), 'Distance_km'].iat[0]
        terminal_distance_dict[terminal] = terminal_distance
    
    terminal_distance_dict = sorted(terminal_distance_dict.items(), key=lambda item: item[1])

    return terminal_distance_dict

# 현재 시점(batch_date) 기준으로 타임 윈도우 최신화
def convert_timewindow_now(timewindow, batch_date):
    timewindow_now = [[0, 0], 0]
    # timewindow의 날짜를 batch_date와 똑같이 맞추었을 때,
    timewindow_now[0][0] = timewindow[0][0].replace(day=batch_date.day)
    timewindow_now[0][1] = timewindow_now[0][0] + timewindow[1]
    timewindow_now[1] = timewindow[1]
    # timewindow보다 batch_date가 같거나 작으면 (batch_date가 timewindow 사이에 있으면)
    if timewindow_now[0][0] <= batch_date:
        if batch_date - timewindow_now[0][0] <= timedelta(hours=1):
            return timewindow_now
        elif (batch_date - timewindow_now[0][0] > timedelta(hours=1)) and (batch_date < timewindow[0][1]):
            timewindow_now[0][0] = timewindow[0][0].replace(
                day=batch_date.day + 1)
            timewindow_now[0][1] = timewindow_now[0][0] + timewindow_now[1]
            return timewindow_now
        else:
            timewindow_now[0][0] = timewindow[0][0].replace(
                day=batch_date.day + 1)
            timewindow_now[0][1] = timewindow_now[0][0] + timewindow_now[1]
            return timewindow_now
            
    else:   # timewindow가 batch_date보다 크면
        if (timewindow_now[0][0] - batch_date >= timedelta(hours=23)) and timewindow_now[0][0].day != 1:    # 1일 -> 0일 X
            timewindow_now[0][0] = timewindow[0][0].replace(
                day=batch_date.day - 1)
            timewindow_now[0][1] = timewindow_now[0][0] + timewindow_now[1]
            return timewindow_now
        elif (timewindow_now[0][1].hour < timewindow_now[0][0].hour) and (timewindow_now[0][1] > (datetime(2023, 5, 8, 0, 0))):
            timewindow_now[0][1] = (datetime(2023, 5, 8, 0, 0))
            return timewindow_now
        else:
            return timewindow_now
        
# 차량 목록 최신화
def get_unique_max_load(vehicles):
    unique_max_load = set(vehicle.max_load for vehicle in vehicles)
    return unique_max_load

In [11]:
############ 함수화 작업 모음 #####################

# 최적의 효율의 차량 선택
def optimal_efficient_vehicle(vehicles, orders_timewindow, unique_max_load, distance_between_other_terminals):
    min_cost = 1000
    final_orders = 0    # 초기화
    final_vehicle = 0
    for max_load in unique_max_load:
        candidate_vehicle = next((vehicle for vehicle in vehicles if (vehicle.dispatched==0) & (vehicle.max_load==max_load)), None)
        if candidate_vehicle == None:
            continue
        candidate_orders = knapsack_max_cbm(orders_timewindow, candidate_vehicle) # 주문 상차
        if len(candidate_orders) == 0:
            continue

        denominator = sum(order.cbm for order in candidate_orders)
        if denominator != 0:
            cost_candidate = candidate_vehicle.fixed_cost + (candidate_vehicle.variable_cost * distance_between_other_terminals) / denominator
        else:
            cost_candidate = 0
        
        if (cost_candidate < min_cost) and (cost_candidate != 0): # 주문들의 cbm 합계가 0인 경우 제외
            min_cost = cost_candidate
            final_orders = candidate_orders
            final_vehicle = candidate_vehicle
    
    return final_orders, final_vehicle

# 물품 개수 제한이 있는 최적의 효율 차량 선택
def optimal_efficient_vehicle_max_items(vehicles, orders_timewindow, unique_max_load, distance_between_terminals, max_items):
    min_cost = 1000
    for max_load in unique_max_load:
        candidate_vehicle = next((vehicle for vehicle in vehicles if (vehicle.dispatched==0) & (vehicle.max_load==max_load)), None)
        if candidate_vehicle == None:
            continue
        candidate_orders = knapsack_max_items_cbm(orders_timewindow, candidate_vehicle, max_items) # 주문 상차
        if len(candidate_orders) == 0:
            continue

        denominator = sum(order.cbm for order in candidate_orders)
        if denominator != 0:
            cost_candidate = candidate_vehicle.fixed_cost + (candidate_vehicle.variable_cost * distance_between_terminals) / denominator
        else:
            cost_candidate = 0
        
        if (cost_candidate < min_cost) and (cost_candidate != 0): # 주문들의 cbm 합계가 0인 경우 제외
            min_cost = cost_candidate
            final_orders = candidate_orders
            final_vehicle = candidate_vehicle

    return final_orders, final_vehicle

# 타임 윈도우 준수하지 못한 경우 배차
# 인근 터미널에서 가져오는 경우는 고려할 필요 없나? (distance_between_terminals)
def batch_time_window_not_honored(orders_timewindow, unique_max_load, distance_between_terminals, max_items):
    
    timewindow_met_or_not = False

    while timewindow_met_or_not == False:
        max_items = max_items - 1   # 최대 물품 개수 1씩 줄이기

        if max_items==0:
            print("더 이상 time window를 만족시킬 수 없습니다. 다음 군집으로.")
            final_orders = 0
            final_vehicle = 0
            break

        # 배차 가능 차량 중, 가장 효율 좋은 차량 선택
        # 최대 적재 가능 물품 수 반영
        final_orders = 0    # 초기화
        final_vehicle = 0
        final_orders, final_vehicle = optimal_efficient_vehicle_max_items(vehicles, orders_timewindow, unique_max_load, distance_between_terminals, max_items)

        # final_orders가 안 바뀔 때 예외처리
        if (final_orders == 0) or (final_vehicle == 0):
            print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 1")
            print(unique_max_load)
            print(final_orders)
            print(final_vehicle)
            break

    return final_orders, final_vehicle

# 인근 터미널에서 차량 가져와서 최종 배차까지
def dispatch_vehicles_nearby_terminals(nearest_terminal_list, orders_timewindow, timewindow_now, batch_date, terminal, cluster, vehicles_all, final_order_table):
    # 배차되는 순간 break되는 반복문                     
    for nearest_terminal in nearest_terminal_list:
        
        # print(nearest_terminal)

        # 해당 인근 터미널의 차량 목록 파악
        # Vehicle.instances = [vehicle for vehicle in vehicles if (vehicle.cur_loc==nearest_terminal[0]) and (vehicle.dispatched==0)]
        vehicle_in_terminals = [vehicle for vehicle in vehicles_all if (vehicle.cur_loc==nearest_terminal[0]) and (vehicle.dispatched==0)]

        if vehicle_in_terminals == []: # 할당할 차량이 없는 경우
            continue    # 다음 터미널로
        else:   # 할당할 차량이 있는 경우
            # 파악된 차량 목록 바탕으로 배차 진행 (전과 동일)
            unique_max_load = list(get_unique_max_load(vehicle_in_terminals))

            # 군집 간 이동거리 확인 (정확한 이동거리는 아님, 추후 정확한 이동거리 계산하여 실험 예정)
            distance_between_terminals = df_od_matrix.query('(origin == @terminal) & (Destination == @cluster)')['Distance_km'].iat[0] + df_od_matrix.query('(origin == @cluster) & (Destination == @terminal)')['Distance_km'].iat[0] + 30
            distance_between_other_terminals = distance_between_terminals + df_od_matrix.query('(origin == @nearest_terminal[0]) & (Destination == @terminal)')['Distance_km'].iat[0]

            # 최적의 효율의 차량 선택
            final_orders, final_vehicle = optimal_efficient_vehicle(
                vehicle_in_terminals, orders_timewindow, unique_max_load, distance_between_other_terminals)

            # final_orders가 안 바뀔 때 예외처리
            if (final_orders == 0) or (final_vehicle == 0):
                print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 2")
                print(unique_max_load)
                print(final_orders)
                print(final_vehicle)
                continue
            
            # 타임 윈도우 준수 여부 판단
            total_time, timewindow_met_or_not, wait_time, total_distance, travel_time, service_time, final_order_table = delivery_time(
                final_orders, df_od_matrix, timewindow_now, batch_date, final_order_table)

            # 타임 윈도우 지켜지지 않았을 경우(timewindow_met_or_not==False) 처리
            max_items = len(final_orders)
            if timewindow_met_or_not == False:
                final_orders, final_vehicle = batch_time_window_not_honored(
                    orders_timewindow, unique_max_load, distance_between_other_terminals, max_items)
            
            # 각 주문의 cbm이 할당된 차량의 최대적재량보다 클 때 continue
            if (final_orders == 0) or (final_vehicle == 0):
                print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 3")
                print(unique_max_load)
                print(final_orders)
                print(final_vehicle)
                continue
            
            return final_orders, final_vehicle, total_time, wait_time, total_distance, travel_time, service_time, final_order_table


# 최종 상차
global sequence
sequence = 1


def final_dispatch(final_orders, final_vehicle, orders_timewindow, total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch):

    global sequence    

    # 최종 주문 테이블 최신화
    for order in final_orders:   # 주문 목록에서 배정된 주문 완료 (delivered=1)
        order.delivered = 1
        condition_order = final_order_table['주문ID'] == order.order_id
        
        final_order_table.loc[condition_order, 'VehicleID'] = final_vehicle.veh_num 
        final_order_table.loc[condition_order, 'Sequence'] = sequence
        sequence += 1
        final_order_table.loc[condition_order, '주문ID_copy'] = float('NaN')
        final_order_table.loc[condition_order, 'BatchDate'] = batch_date
        final_order_table.loc[condition_order, '착지ID'] = order.arrival_id
        final_order_table.loc[condition_order, 'Delivered'] = 'YES'

    orders_timewindow = [order for order in orders_timewindow if order.delivered == 0]
    
    # 최종 차량 테이블 최신화
    condition = final_vehicle_dispatch['VehNum'] == final_vehicle.veh_num

    final_vehicle_dispatch.loc[condition, 'Count'] += len(final_orders)
    final_vehicle_dispatch.loc[condition, 'Volume'] += sum(order.cbm_former for order in final_orders)
    final_vehicle_dispatch.loc[condition, 'TravelDistance'] += total_distance
    final_vehicle_dispatch.loc[condition, 'WorkTime'] += (wait_time + travel_time + service_time)
    final_vehicle_dispatch.loc[condition, 'TravelTime'] += travel_time
    final_vehicle_dispatch.loc[condition, 'ServiceTime'] += service_time

    # WaitingTime, FixedCost, VariableCost, TotalCost 값 동시에 수정
    final_vehicle_dispatch.loc[condition, ['WaitingTime', 'FixedCost', 'VariableCost', 'TotalCost']] += wait_time, final_vehicle.fixed_cost, final_vehicle.variable_cost * total_distance, final_vehicle.fixed_cost + final_vehicle.variable_cost * total_distance
    
    # 배정된 주문 배송에 걸리는 시간 계산
    if total_time > 0:
        final_vehicle.dispatched = 1
    elif total_time > 360:
        final_vehicle.dispatched = 2
    elif total_time > 720:
        final_vehicle.dispatched = 3

    return orders_timewindow

# 데드라인 얼마 안 남은 경우와 배치 마지막날인 경우 사용되는 배차 알고리즘


def full_dispatch_algorithm_deadline(orders_timewindow, vehicle_in_terminals, batch_date, unique_dates, timewindow_threshold, timewindow, timewindow_now, terminal, cluster, distance_between_terminals, time_to_cluster, vehicles_all, final_order_table, final_vehicle_dispatch):
    
    # 주문 데드라인까지 얼마 남지 않은 주문이 포함된 경우 or 배치 마지막 날인 경우
    while orders_shipped_quickly(orders_timewindow, batch_date) or ((unique_dates.index(batch_date) > (len(unique_dates)-5)) and (len(orders_timewindow) > 0)):

        ## 타임 윈도우 시작 시간과 차이가 없는 경우
        temp = (timewindow_now[0][0] - batch_date) % timedelta(hours=24)
        if (temp <= timedelta(hours=timewindow_threshold)) or (temp >= timedelta(hours=24-(6-timewindow_threshold))):  # 타임 윈도우 시작 시간에서 배치 시간을 뺀 시간이 5시간 이하이면

            # 배차 가능한 차량 유형 초기화
            vehicle_in_terminals = [vehicle for vehicle in vehicle_in_terminals if vehicle.dispatched==0]

            ## 현재 터미널에 가용 차량이 없는 경우 처리
            if vehicle_in_terminals == []:
                
                print("현재 터미널에 가용 차량이 없습니다. 1")

                # 현재 터미널에서 가장 가까운 터미널 알아내기
                nearest_terminal_list = terminal_distance_dict_total[terminal]

                # 인근 터미널 돌아가면서 할당할 차량 있으면 배차까지
                final_orders, final_vehicle, total_time, wait_time, total_distance, travel_time, service_time, final_order_table = dispatch_vehicles_nearby_terminals(
                    nearest_terminal_list, orders_timewindow, timewindow_now, batch_date, terminal, cluster, vehicles_all, final_order_table)
                
                # 최종 상차
                orders_timewindow = final_dispatch(
                    final_orders, final_vehicle, orders_timewindow, total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch)

                # 처리중인 터미널로 복귀
                final_vehicle.cur_loc = terminal
                
                continue

            unique_max_load = list(get_unique_max_load(vehicle_in_terminals))

            if len(unique_max_load) == 0:
                print("현재 터미널에 배차 가능한 차량이 없습니다")
                ## 현재 처리 중인 터미널에서 가장 가까운 인근 터미널의 차량 현황 파악 후 가져오기
                break

            # 배차 가능 차량 중, 가장 효율 좋은 차량 선택
            final_orders, final_vehicle = optimal_efficient_vehicle(
                vehicle_in_terminals, orders_timewindow, unique_max_load, distance_between_terminals)

            # final_orders가 안 바뀔 때 예외처리
            if (final_orders == 0) or (final_vehicle == 0):
                print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 4")
                print(unique_max_load)
                print(final_orders)
                print(final_vehicle)
                break

            # 총 시간 계산하며 time window 충족 여부 판단
            total_time, timewindow_met_or_not, wait_time, total_distance, travel_time, service_time, final_order_table = delivery_time(
                final_orders, df_od_matrix, timewindow_now, batch_date, final_order_table)
            
            # 타임 윈도우 지켜지지 않았을 경우(timewindow_met_or_not==False) 처리
            max_items = len(final_orders)
            if timewindow_met_or_not == False:
                final_orders, final_vehicle = batch_time_window_not_honored(orders_timewindow, unique_max_load, distance_between_terminals, max_items)

            # final_orders가 안 바뀔 때 예외처리
            if (final_orders == 0) or (final_vehicle == 0):
                print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 5")
                print(unique_max_load)
                print(final_orders)
                print(final_vehicle)
                print(wait_time)
                print(total_time)
                print(timewindow_now)
                print(timewindow)
                print(time_to_cluster)
                break
            
            # 최종 상차
            orders_timewindow = final_dispatch(
                final_orders, final_vehicle, orders_timewindow, total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch)
        
        else:   # 타임 윈도우 시작 시간에서 배치 시간을 뺀 시간이 5시간을 넘으면

            if time_to_cluster > 150:
                # 배차 가능한 차량 유형 초기화
                vehicle_in_terminals = [vehicle for vehicle in vehicle_in_terminals if vehicle.dispatched==0]

                ## 현재 터미널에 가용 차량이 없는 경우 처리
                if vehicle_in_terminals == []:
                    print("현재 터미널에 가용 차량이 없습니다. 2")

                    # 현재 터미널에서 가장 가까운 터미널 알아내기
                    nearest_terminal_list = terminal_distance_dict_total[terminal]
                    # print(nearest_terminal_list)

                    # 인근 터미널 돌아가면서 할당할 차량 있으면 배차까지
                    final_orders, final_vehicle, total_time, wait_time, total_distance, travel_time, service_time, final_order_table = dispatch_vehicles_nearby_terminals(
                        nearest_terminal_list, orders_timewindow, timewindow_now, batch_date, terminal, cluster, vehicles_all, final_order_table)
                    
                    # 최종 상차
                    orders_timewindow = final_dispatch(
                        final_orders, final_vehicle, orders_timewindow, total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch)

                    # 처리중인 터미널로 복귀
                    final_vehicle.cur_loc = terminal
                    
                    continue

                unique_max_load = list(get_unique_max_load(vehicle_in_terminals))

                if len(unique_max_load) == 0:
                    print("현재 터미널에 배차 가능한 차량이 없습니다")
                    ## 현재 처리 중인 터미널에서 가장 가까운 인근 터미널의 차량 현황 파악 후 가져오기
                    break

                # 배차 가능 차량 중, 가장 효율 좋은 차량 선택
                final_orders, final_vehicle = optimal_efficient_vehicle(
                    vehicle_in_terminals, orders_timewindow, unique_max_load, distance_between_terminals)

                # final_orders가 안 바뀔 때 예외처리
                if (final_orders == 0) or (final_vehicle == 0):
                    print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 6")
                    print(unique_max_load)
                    print(final_orders)
                    print(final_vehicle)
                    break

                # 총 시간 계산하며 time window 충족 여부 판단
                total_time, timewindow_met_or_not, wait_time, total_distance, travel_time, service_time, final_order_table = delivery_time(
                    final_orders, df_od_matrix, timewindow_now, batch_date, final_order_table)
                
                # 타임 윈도우 지켜지지 않았을 경우(timewindow_met_or_not==False) 처리
                max_items = len(final_orders)
                if timewindow_met_or_not == False:
                    final_orders, final_vehicle = batch_time_window_not_honored(orders_timewindow, unique_max_load, distance_between_terminals, max_items)

                # final_orders가 안 바뀔 때 예외처리
                if (final_orders == 0) or (final_vehicle == 0):
                    print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 7")
                    print(unique_max_load)
                    print(final_orders)
                    print(final_vehicle)
                    print(wait_time)
                    print(total_time)
                    print(timewindow_now)
                    print(timewindow)
                    break
                
                # 최종 상차
                orders_timewindow = final_dispatch(
                    final_orders, final_vehicle, orders_timewindow, total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch)

            else:
                break

# 데드라인으로부터 여유있을 때 사용되는 배차 알고리즘
def full_dispatch_algorithm(orders_timewindow, vehicle_in_terminals, batch_date, unique_dates, timewindow_threshold, timewindow,
                            timewindow_now, terminal, cluster, distance_between_terminals, time_to_cluster, vehicles_all, final_order_table, final_vehicle_dispatch):
    ## 주문 데드라인까지 여유있는 경우 (주문 데이터의 적재량 합이 배차된 차량의 최대적재량을 넘기면)
    while comp_orderload_vehmaxload(orders_timewindow):

        ## 타임 윈도우 시작 시간과 차이가 없는 경우
        temp = (timewindow_now[0][0] - batch_date) % timedelta(hours=24)
        if (temp <= timedelta(hours=timewindow_threshold)) or (temp >= timedelta(hours=24-(6-timewindow_threshold))):  # 타임 윈도우 시작 시간에서 배치 시간을 뺀 시간이 5시간 이하이면

            # 배차 가능한 차량 유형 초기화
            vehicle_in_terminals = [vehicle for vehicle in vehicle_in_terminals if vehicle.dispatched==0]

            ## 현재 터미널에 가용 차량이 없는 경우 처리
            if vehicle_in_terminals == []:
                print("현재 터미널에 가용 차량이 없습니다. 3")

                # 현재 터미널에서 가장 가까운 터미널 알아내기
                nearest_terminal_list = terminal_distance_dict_total[terminal]

                # 인근 터미널 돌아가면서 할당할 차량 있으면 배차까지
                final_orders, final_vehicle, total_time, wait_time, total_distance, travel_time, service_time, final_order_table = dispatch_vehicles_nearby_terminals(
                    nearest_terminal_list, orders_timewindow, timewindow_now, batch_date, terminal, cluster, vehicles_all, final_order_table)
                
                # 최종 상차
                orders_timewindow = final_dispatch(
                    final_orders, final_vehicle, orders_timewindow, total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch)

                # 처리중인 터미널로 복귀
                final_vehicle.cur_loc = terminal
                
                continue

            unique_max_load = list(get_unique_max_load(vehicle_in_terminals))

            if len(unique_max_load) == 0:
                print("현재 터미널에 배차 가능한 차량이 없습니다")
                ## 현재 처리 중인 터미널에서 가장 가까운 인근 터미널의 차량 현황 파악 후 가져오기
                break

            # 배차 가능 차량 중, 가장 효율 좋은 차량 선택
            final_orders, final_vehicle = optimal_efficient_vehicle(
                vehicle_in_terminals, orders_timewindow, unique_max_load, distance_between_terminals)

            # final_orders가 안 바뀔 때 예외처리
            if (final_orders == 0) or (final_vehicle == 0):
                print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 8")
                print(unique_max_load)
                print(final_orders)
                print(final_vehicle)
                break

            # 총 시간 계산하며 time window 충족 여부 판단
            total_time, timewindow_met_or_not, wait_time, total_distance, travel_time, service_time, final_order_table = delivery_time(
                final_orders, df_od_matrix, timewindow_now, batch_date, final_order_table
            )
            
            # 타임 윈도우 지켜지지 않았을 경우(timewindow_met_or_not==False) 처리
            max_items = len(final_orders)
            if timewindow_met_or_not == False:
                final_orders, final_vehicle = batch_time_window_not_honored(orders_timewindow, unique_max_load, distance_between_terminals, max_items)
            
            # print("이동 시간: ", total_time)

            # final_orders가 안 바뀔 때 예외처리
            if (final_orders == 0) or (final_vehicle == 0):
                print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 9")
                print(unique_max_load)
                print(final_orders)
                print(final_vehicle)
                print(wait_time)
                print(total_time)
                print(timewindow_now)
                print(timewindow)
                break
            
            # 최종 상차
            orders_timewindow = final_dispatch(final_orders, final_vehicle, orders_timewindow,
                                               total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch)
            
        ## 타임 윈도우 시작 시간까지 많이 남은 경우 
        else:   # 타임 윈도우 시작 시간에서 배치 시간을 뺀 시간이 5시간을 넘으면

            if time_to_cluster > 180:
                # 배차 가능한 차량 유형 초기화
                vehicle_in_terminals = [vehicle for vehicle in vehicle_in_terminals if vehicle.dispatched==0]

                ## 현재 터미널에 가용 차량이 없는 경우 처리
                if vehicle_in_terminals == []:
                    print("현재 터미널에 가용 차량이 없습니다. 4")

                    # 현재 터미널에서 가장 가까운 터미널 알아내기
                    nearest_terminal_list = terminal_distance_dict_total[terminal]

                    # 인근 터미널 돌아가면서 할당할 차량 있으면 배차까지
                    final_orders, final_vehicle, total_time, wait_time, total_distance, travel_time, service_time, final_order_table = dispatch_vehicles_nearby_terminals(
                        nearest_terminal_list, orders_timewindow, timewindow_now, batch_date, terminal, cluster, vehicles_all, final_order_table)
                    
                    # 최종 상차
                    orders_timewindow = final_dispatch(
                        final_orders, final_vehicle, orders_timewindow, total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch)
                    # 처리중인 터미널로 복귀
                    final_vehicle.cur_loc = terminal
                    
                    continue

                unique_max_load = list(get_unique_max_load(vehicle_in_terminals))

                if len(unique_max_load) == 0:
                    print("현재 터미널에 배차 가능한 차량이 없습니다")
                    ## 현재 처리 중인 터미널에서 가장 가까운 인근 터미널의 차량 현황 파악 후 가져오기
                    break

                # 배차 가능 차량 중, 가장 효율 좋은 차량 선택
                final_orders, final_vehicle = optimal_efficient_vehicle(
                    vehicle_in_terminals, orders_timewindow, unique_max_load, distance_between_terminals)

                # final_orders가 안 바뀔 때 예외처리
                if (final_orders == 0) or (final_vehicle == 0):
                    print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 10")
                    print(unique_max_load)
                    print(final_orders)
                    print(final_vehicle)
                    break

                # 총 시간 계산하며 time window 충족 여부 판단
                total_time, timewindow_met_or_not, wait_time, total_distance, travel_time, service_time, final_order_table = delivery_time(
                    final_orders, df_od_matrix, timewindow_now, batch_date, final_order_table)
                
                # 타임 윈도우 지켜지지 않았을 경우(timewindow_met_or_not==False) 처리
                max_items = len(final_orders)
                if timewindow_met_or_not == False:
                    final_orders, final_vehicle = batch_time_window_not_honored(orders_timewindow, unique_max_load, distance_between_terminals, max_items)

                # final_orders가 안 바뀔 때 예외처리
                if (final_orders == 0) or (final_vehicle == 0):
                    print("뭐가 문제일까? 배차할 차량이 없는 게 맞을까? 11")
                    print(unique_max_load)
                    print(final_orders)
                    print(final_vehicle)
                    print(wait_time)
                    print(total_time)
                    print(timewindow_now)
                    print(timewindow)
                    break
                
                # 최종 상차
                orders_timewindow = final_dispatch(
                    final_orders, final_vehicle, orders_timewindow, total_distance, total_time, travel_time, service_time, wait_time, batch_date, final_order_table, final_vehicle_dispatch)

            else:
                break

In [12]:
from collections import defaultdict

# 배치 생성 (6시간 단위)
Order.instances = orders
unique_dates = sorted(list(Order.get_unique_dates()))   # 배치 고유값 가져오기 (배치 생성 시점)
for i in range(1, 5):   # 7일차 배치 추가 생성
    unique_dates.append(unique_dates[-1] + timedelta(hours=6))
unique_dates = tuple(unique_dates)
print(len(unique_dates))

# 주문 시간 순으로 정렬
orders.sort(key=lambda x: x.date)

# timewindow 스레시홀드
timewindow_threshold = 5

# 배치마다 처리
for batch_date in sorted(list(unique_dates)):
    
    # 최종 주문 테이블) 최신화 (final_order_table)
    batch_temp = [order for order in orders if (order.date==batch_date)]
    for order in batch_temp:
        condition_order = final_order_table['주문ID'] == order.order_id
        
        final_order_table.loc[final_order_table['주문ID'] == order.order_id, '착지ID'] = order.item_loc # 주문의 터미널ID 표기
        final_order_table.loc[final_order_table['주문ID'] == order.order_id, '주문ID_copy'] = float('NaN')
        
    final_order_table_copy = final_order_table.copy()
    
    order_temp = 0
    for index, row in final_order_table_copy.iterrows():
        if pd.isna(row.DepartureTime) == False:
            if row.DepartureTime > batch_date:
                row.DepartureTime = float('NaN')
                row.WaitingTime = float('NaN')
                row.ServiceTime = float('NaN')
                row.ArrivalTime = float('NaN')
                for order in orders:
                    if order.order_id == row['주문ID']:
                        order_temp = order
                        break
                row['착지ID'] = order_temp.item_loc
                row.Delivered = 'NO'
                
    # 최종 주문 테이블 파일화
    var = "FinalTable/제발_5/final_order_table_{}.csv".format(batch_date)
    final_order_table_copy.to_csv(path_or_buf=var, index=False)

    print(batch_date)
    print("총 처리한 주문 수:", len([order for order in orders if order.delivered==1]))

    # 모든 차량들의 이동 시간 고려하여 배차 가능 여부 판단
    # 좀 더 빡빡하게 짤 수 있도록 고민해야 함!!!
    for vehicle in vehicles:
        if vehicle.dispatched == 1:
            vehicle.dispatched = 0
        elif vehicle.dispatched == 2:
            vehicle.dispatched = 1
        elif vehicle.dispatched == 3:
            vehicle.dispatched = 2

    batch = [order for order in orders if ((order.date<=batch_date) & (order.delivered==0))]

    # 터미널마다 반복
    for terminal in list(df_terminals['ID'].unique()):

        # 해당 터미널에 상주하는 차량만 가져오기
        # Vehicle.instances = [vehicle for vehicle in vehicles if (vehicle.cur_loc==terminal) and (vehicle.dispatched==0)]
        vehicle_in_terminals = [vehicle for vehicle in vehicles if (vehicle.cur_loc==terminal) and (vehicle.dispatched==0)]

        # 군집마다 반복
        # 군집 순서 결정 기준 세울 필요 있을 듯 (랜덤하게 or 순서 그대로 or 주문 많은 순서대로)
        for cluster in list(df_final['가장 가까운 터미널'].unique()):
            # print("군집 터미널: ", cluster)

            # 배치에서 해당 터미널의 특정 군집의 주문 데이터 추출
            orders_cluster = [order for order in batch if ((order.item_loc==terminal) and (order.cluster==cluster) and (order.delivered==0))]
            # print("군집 내 주문 수: ", len(orders_cluster))

            if len(orders_cluster)==0: # 군집에 주문 데이터가 없으면 pass
                continue

            # 군집 내 주문 데이터의 타임 윈도우 고유값 가져오기
            unique_timewindows = get_unique_timewindows_combine(orders_cluster)    
            unique_timewindows = sorted(unique_timewindows, key=lambda x: x[0][1])  # 타임 윈도우 끝나는 시점 기준 오름차순 정렬 (먼저 끝나는 주문 먼저 처리하기 위함)

            # 해당 군집의 터미널까지 가는데 걸리는 시간 계산 (대략적인 이동시간 계산을 위함)
            time_to_cluster = df_od_matrix.query('(origin == @terminal) & (Destination == @cluster)')['Time_minute'].iat[0]

            # 군집 간 이동거리 확인 (정확한 이동거리는 아님, 추후 정확한 이동거리 계산하여 실험 예정)
            distance_between_terminals = df_od_matrix.query('(origin == @terminal) & (Destination == @cluster)')['Distance_km'].iat[0] + df_od_matrix.query('(origin == @cluster) & (Destination == @terminal)')['Distance_km'].iat[0] + 30

            # 타임 윈도우 겹치는 친구들끼리 뭉쳐서 dictionary 형성
            timewindow_now_dict = {}
            for timewindow in unique_timewindows:
                timewindow_now = convert_timewindow_now(timewindow, batch_date)
                timewindow_now_dict[timewindow] = timewindow_now
                if timewindow_now[0][0] < (sorted(list(unique_dates))[-1] + timedelta(hours=6)) < timewindow_now[0][1]:
                    timewindow_now[0][1] = (sorted(list(unique_dates))[-1] + timedelta(hours=6))
                    print(timewindow_now)
            timewindow_now_to_group = defaultdict(list)
            for key, value in timewindow_now_dict.items():
                value[0] = tuple(value[0])
                timewindow_now_to_group[tuple(value)].append(key)  # Use tuple instead of list

            ## 타임 윈도우별로 처리
            # for timewindow in unique_timewindows:   # 타임 윈도우별로 묶기
            for timewindow_now, timewindow in timewindow_now_to_group.items():

                # 배치시간 고려하여 특정 타임윈도우는 패스
                # 배치의 날짜를 제외한 시간으로부터 타임 윈도우 시작 시간까지 8~23시간 사이는 제외
                # 이 때 어떤 타임 윈도우만 살아남나?
                temp = (timewindow_now[0][0] - batch_date) % timedelta(hours=24)
                if (temp >= timedelta(hours=12)) and (temp < timedelta(hours=24-(6-timewindow_threshold))):
                    continue

                # 해당 타임윈도우의 배차되지 않은 주문 데이터 모으기
                orders_timewindow = []
                for timewindow_element in timewindow:
                    orders_timewindow += [order for order in orders_cluster if (order.timewindow==timewindow_element[0]) and (order.delivered==0)]
                # print("해당 타임 윈도우 주문 개수:", len(orders_timewindow))
                orders_timewindow = sorted(orders_timewindow, key=lambda x: x.date) # 날짜순으로 정렬

                # 데드라인 얼마 안 남은 경우와 배치 마지막날인 경우 사용되는 배차 알고리즘
                full_dispatch_algorithm_deadline(orders_timewindow, vehicle_in_terminals, batch_date, unique_dates, timewindow_threshold, timewindow, timewindow_now, terminal, cluster, distance_between_terminals, time_to_cluster, vehicles, final_order_table, final_vehicle_dispatch)
                
                # 데드라인으로부터 여유있을 때 사용되는 배차 알고리즘
                full_dispatch_algorithm(orders_timewindow, vehicle_in_terminals, batch_date, unique_dates, timewindow_threshold,
                                        timewindow, timewindow_now, terminal, cluster, distance_between_terminals, time_to_cluster, vehicles, final_order_table, final_vehicle_dispatch)
                
var = "FinalTable/제발_5/final_order_table_last.csv"
final_order_table.to_csv(path_or_buf=var, index=False)


28
2023-05-01 00:00:00
총 처리한 주문 수: 0


KeyboardInterrupt: 

In [ ]:
import pickle
temp = [order for order in orders if order.delivered==1]

with open("order2_3_2_3버전.pickle","wb") as f:
    pickle.dump(temp, f) # 위에서 생성한 리스트를 list.pickle로 저장
print(len(temp))
print(len(orders))

In [ ]:
final_vehicle_dispatch.to_csv(
    path_or_buf='FinalTable/제발_5/final_vehicle_dispatch.csv', index=False)
final_order_table.to_csv(path_or_buf='final_order_table_1.csv', index=False)

In [ ]:
# # pickle 파일 불러오기
# with open("order2_3_2_3버전.pickle", "rb") as fi:
#   test = pickle.load(fi) # list.pickle 읽어서 출력 -> list 잘 불러와졌다.
# print(len(test))